In [64]:
import jax_sbgeom
import jax.numpy as jnp
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import h5py 
import numpy as onp

In [93]:
def generate_coil_artefacts(filename, output_prefix, output_name):
    with h5py.File(filename) as f:
        positions = f['Dataset1'].__array__()

    from coils.test_coils import data_input
    
    input_filename = data_input/ f"{output_name}_input.npy"
    
    onp.save(input_filename, positions)

    from coils.test_coils import _get_all_discrete_coils, _get_positions, _get_tangents, _data_file_to_data_output

    # discrete tests:
    # =================
    coil_list_discrete=  _get_all_discrete_coils(input_filename)

    # position
    discrete_pos_data_file = _data_file_to_data_output(input_filename, "_discrete_coil_position")
    onp.save( discrete_pos_data_file, _get_positions(coil_list_discrete))

    # tangent
    discrete_tan_data_file = _data_file_to_data_output(input_filename, "_discrete_coil_tangent")
    onp.save( discrete_tan_data_file, _get_tangents(coil_list_discrete))

    # fourier tests:
    # =================
    from coils.test_coils import _get_all_fourier_coils
    coil_list_fourier=  _get_all_fourier_coils(input_filename)

    # position
    fourier_pos_data_file = _data_file_to_data_output(input_filename, "_fourier_coil_position")
    onp.save( fourier_pos_data_file, _get_positions(coil_list_fourier))

    # tangent   
    fourier_tan_data_file = _data_file_to_data_output(input_filename, "_fourier_coil_tangent")
    onp.save( fourier_tan_data_file, _get_tangents(coil_list_fourier))

    # Finite Size Tests:
    from coils.test_coils import _get_all_finite_sizes
    
    onp.save( _data_file_to_data_output(input_filename, "_discrete_coil_finite_size"), _get_all_finite_sizes(coil_list_discrete))
    onp.save( _data_file_to_data_output(input_filename, "_fourier_coil_finite_size"), _get_all_finite_sizes(coil_list_fourier))

    # Mesh tests:

    from coils.test_coils import _get_coil_mesh
    onp.save(_data_file_to_data_output(input_filename, "_discrete_coil_mesh"), _get_coil_mesh(coil_list_discrete))
    onp.save(_data_file_to_data_output(input_filename, "_fourier_coil_mesh"), _get_coil_mesh(coil_list_fourier))

    # Conversion tests:

    from coils.test_coils import _get_fourier_transformation

    onp.save(_data_file_to_data_output(input_filename, "_fourier_coil_transformation"), _get_fourier_transformation(coil_list_discrete, None))
    onp.save(_data_file_to_data_output(input_filename, "_fourier_coil_transformation_nft11"), _get_fourier_transformation(coil_list_discrete, 11))


filename = "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5"
generate_coil_artefacts(filename , "data/coils/", "helias5")

filename_h4 = "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS4_coils_all.h5"
generate_coil_artefacts(filename_h4 , "data/coils/", "helias4")

In [99]:
def generate_flux_surface_artefacts(filename, output_name, use_sbgeom_old = True):
    import shutil
    from flux_surfaces.test_flux_surface_base_data import DATA_INPUT_FLUX_SURFACES, _data_file_to_data_output
    if use_sbgeom_old:
        import StellBlanket.SBGeom as SBGeom
        sbgeom_fs = SBGeom.VMEC.Flux_Surfaces_From_HDF5(filename)
    

    shutil.copyfile(filename, f"data/flux_surfaces/{output_name}_input.h5")
    
    input_filename = DATA_INPUT_FLUX_SURFACES / f"{output_name}_input.h5"

    input_flux_surface = jax_sbgeom.flux_surfaces.FluxSurface.from_hdf5(input_filename)

    from flux_surfaces.test_flux_surface_base_data import _get_positions_jsb, _get_positions_sbgeom, _sampling_grid, _1d_sampling_grid
    if use_sbgeom_old:
        positions = _get_positions_sbgeom(input_flux_surface, sbgeom_fs, _1d_sampling_grid)
    else:
        positions = _get_positions_jsb(input_flux_surface, _sampling_grid)
    onp.save(_data_file_to_data_output(input_filename, "_position"), positions)        

    from flux_surfaces.test_flux_surface_base_data import _get_normals_jsb, _get_normals_sbgeom
    if use_sbgeom_old:
        normals = _get_normals_sbgeom(input_flux_surface, sbgeom_fs, _1d_sampling_grid)
    else:
        normals = _get_normals_jsb(input_flux_surface, _sampling_grid)
    onp.save(_data_file_to_data_output(input_filename, "_normal"), normals)
    
    from flux_surfaces.test_flux_surface_base_data import _get_principal_curvatures_jsb, _get_principal_curvatures_sbgeom
    if use_sbgeom_old:
        curvatures = _get_principal_curvatures_sbgeom(input_flux_surface, sbgeom_fs, _1d_sampling_grid)
    else:
        curvatures = _get_principal_curvatures_jsb(input_flux_surface, _sampling_grid)
    onp.save(_data_file_to_data_output(input_filename, "_curvature"), curvatures)
    
    # Meshing 

    from flux_surfaces.test_flux_surface_base_data import _get_meshing_surface_jsb, _get_meshing_surface_sbgeom, n_theta_n_phi
    if use_sbgeom_old:
        mesh = _get_meshing_surface_sbgeom(input_flux_surface, sbgeom_fs, 1.0,  "half_module", *n_theta_n_phi())
    else:
        mesh = _get_meshing_surface_jsb(input_flux_surface, 1.0, "half_module", *n_theta_n_phi())
    
    onp.save(_data_file_to_data_output(input_filename, "_meshing_halfmod_position"), mesh[0])
    onp.save(_data_file_to_data_output(input_filename, "_meshing_halfmod_connectivity"), mesh[1])

    if use_sbgeom_old:
        mesh = _get_meshing_surface_sbgeom(input_flux_surface, sbgeom_fs, 1.0,  "full", *n_theta_n_phi())
    else:
        mesh = _get_meshing_surface_jsb(input_flux_surface, 1.0, "full", *n_theta_n_phi())
    
    onp.save(_data_file_to_data_output(input_filename, "_meshing_full_position"), mesh[0])
    onp.save(_data_file_to_data_output(input_filename, "_meshing_full_connectivity"), mesh[1])

    from flux_surfaces.test_flux_surface_base_data import _get_all_cases_jsb, _get_all_cases_sbgeom

    if use_sbgeom_old:
        meshes = _get_all_cases_sbgeom(sbgeom_fs)
    else:
        meshes = _get_all_cases_jsb(input_flux_surface)

    onp.save(_data_file_to_data_output(input_filename, "_tetrahedral_all_position"), meshes[0])  # Dummy file to indicate completion
    onp.save(_data_file_to_data_output(input_filename, "_tetrahedral_all_connectivity"), meshes[1])  # Dummy file to indicate completion

    from flux_surfaces.test_flux_surface_base_data import _get_watertight_mesh_jsb, _get_watertight_mesh_sbgeom
    s_values = jnp.linspace(0.1, 1.0, 5)
    
    watertight_mesh = _get_watertight_mesh_jsb(input_flux_surface, s_values, 0.2 * jnp.pi, *n_theta_n_phi())

    onp.save(_data_file_to_data_output(input_filename, "_watertight_position"), watertight_mesh[0])
    onp.save(_data_file_to_data_output(input_filename, "_watertight_connectivity"), onp.concatenate(watertight_mesh[1], axis=0))

    # Extension
    from flux_surfaces.test_flux_surface_base_extension import extension_types, extension_type_to_name, _extended_1d_sampling_grid, _extended_1d_sampling_grid_stop_1, _extended_sampling_grid

    def extension_type_to_sbgeom_extension(sbgeom_fs, extension_type):
        print(extension_type)
        if extension_type == "normal":
            return SBGeom.Flux_Surfaces_Normal_Extended(sbgeom_fs)
        elif extension_type == "normal_no_phi":
            return SBGeom.Flux_Surfaces_Normal_Extended_No_Phi(sbgeom_fs)
        elif extension_type == "normal_constant_phi":
            return SBGeom.Flux_Surfaces_Normal_Extended_Constant_Phi(sbgeom_fs)
        else:
            raise ValueError(f"Unknown extension type: {extension_type}")

    for extension_type in extension_types():
        if use_sbgeom_old:
            sbgeom_extension = extension_type_to_sbgeom_extension(sbgeom_fs, extension_type_to_name(extension_type))
            extension_pos = _get_positions_sbgeom(input_flux_surface, sbgeom_extension, _extended_1d_sampling_grid)
        else:            
            fs_jax = extension_type.from_hdf5(input_filename)
            onp.testing.assert_allclose( _get_positions_jsb(fs_jax, _extended_sampling_grid), extension_pos)
        onp.save(_data_file_to_data_output(input_filename, f"_{extension_type_to_name(extension_type)}_position"), extension_pos)

        fs_jax = extension_type.from_hdf5(input_filename)
        onp.save(_data_file_to_data_output(input_filename, f"_{extension_type_to_name(extension_type)}_normals"), _get_normals_jsb(fs_jax, _extended_sampling_grid))
        onp.save(_data_file_to_data_output(input_filename, f"_{extension_type_to_name(extension_type)}_curvatures"), _get_principal_curvatures_jsb(fs_jax, _extended_sampling_grid))
    

filename_fs = "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4"
generate_flux_surface_artefacts(filename_fs, "helias5")

filename_fs4 = "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias4_vmec.nc4"
generate_flux_surface_artefacts(filename_fs4, "helias4")

normal
normal_no_phi
normal_constant_phi
normal
normal_no_phi
normal_constant_phi
